In [10]:
# --- Instalar y configurar Kaggle CLI ---
!pip -q install kaggle



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [11]:
import os, json, zipfile, shutil

# Asume que kaggle.json ya está en la ruta local del proyecto

kaggle_json_path = "./kaggle.json"

kaggle_dir = os.path.expanduser("~/.kaggle")

os.makedirs(kaggle_dir, exist_ok=True)

shutil.copy(kaggle_json_path, os.path.join(kaggle_dir, "kaggle.json"))

os.chmod(os.path.join(kaggle_dir, "kaggle.json"), 0o600)


!kaggle --version

print("Kaggle configurado")

Kaggle API 1.7.4.5
Kaggle configurado
Kaggle configurado


In [12]:
# --- Descargar el dataset Fruits-360 desde Kaggle ---
# Nota: '-p /content' define la carpeta destino; '--unzip' descomprime.
!kaggle datasets download -d moltean/fruits -p /content --unzip

Dataset URL: https://www.kaggle.com/datasets/moltean/fruits
License(s): CC-BY-SA-4.0


[Errno 13] Permission denied: '/content'


In [ ]:
import os


dataset_base = "./ProyectoFinal/Dataset_Filtrado"
if not os.path.exists(dataset_base):
    dataset_base = "ProyectoFinal/Dataset_Filtrado"
if not os.path.exists(dataset_base):
    dataset_base = "./Dataset_Filtrado"
if not os.path.exists(dataset_base):
    dataset_base = "Dataset_Filtrado"

if not os.path.exists(dataset_base):
    raise FileNotFoundError(f"No se encontró la carpeta del dataset en ninguna ruta esperada. Ruta probada: {dataset_base}")

carpetas = [d for d in os.listdir(dataset_base) if os.path.isdir(os.path.join(dataset_base, d))]
print("Carpetas encontradas en Dataset_Filtrado:", carpetas)

# Elige 'train' o 'test' si existen, si no muestra advertencia
if "train" in carpetas:
    dataset_origen = os.path.join(dataset_base, "train")
elif "test" in carpetas:
    dataset_origen = os.path.join(dataset_base, "test")
else:
    dataset_origen = None

assert dataset_origen, "No se encontró la carpeta 'train' o 'test' en Dataset_Filtrado."
print("Carpeta localizada:", dataset_origen)
print("Ejemplo de clases:", os.listdir(dataset_origen)[:10])

Carpetas encontradas en Dataset_Filtrado: ['test', 'train']
Carpeta localizada: ./Dataset_Filtrado/train
Ejemplo de clases: ['Pear Forelle 1', 'Kiwi 1', 'Apple Red 1', 'Cocos 1', 'Cantaloupe 2', 'Peach 2', 'Granadilla 1', 'Banana 1', 'Maracuja 1', 'Pineapple Mini 1']


In [19]:
import os, shutil, re

# 1) Muestra cómo se llaman realmente tus clases

print("Ejemplo de clases en Training:")

print(sorted(os.listdir(dataset_origen))[:60])  # dataset_origen ya lo tenías

# 2) Palabras clave a buscar (en minúsculas) -> sinónimos

wish = {

    "apple red 1": ["apple red 1"],

    "banana": ["banana"],

    "melon 2": ["cantaloupe 2","cantaloupe","melon"],

    "cocos": ["cocos","coconut"],

    "granadilla": ["granadilla"],

    "kiwi": ["kiwi"],

    "maracuja": ["maracuja"],

    "peach": ["peach"],

    "pear forelle": ["pear forelle","forelle","pear"],

    "pineapple": ["pineapple","piña","pina"]

}

# 3) Normalizador

def norm(s): return re.sub(r"\s+"," ",s.strip().lower())

# 4) Índice de carpetas reales

classes_real = os.listdir(dataset_origen)

classes_norm = {norm(c): c for c in classes_real}  # norm -> original

# 5) Buscar mejor match por “contiene”

def find_match(keywords):

    kws = [norm(k) for k in keywords]

    # prioridad: coincidencia completa; si no, contiene

    for k in kws:

        if k in classes_norm:

            return classes_norm[k]

    for k in kws:

        for cn, original in classes_norm.items():

            if k in cn:

                return original

    return None

# Ajusta la ruta destino para entorno local

destino_base = "./ProyectoFinal"

dataset_destino = os.path.join(destino_base, "Dataset_Filtrado")

os.makedirs(dataset_destino, exist_ok=True)

copiadas, faltantes = [], []

for esp, keywords in wish.items():

    found = find_match(keywords)

    if found and os.path.isdir(os.path.join(dataset_origen, found)):

        src = os.path.join(dataset_origen, found)

        dst = os.path.join(dataset_destino, found)

        shutil.copytree(src, dst, dirs_exist_ok=True)

        copiadas.append((esp, found))

        print(f"Copiada: '{esp}' -> '{found}'")

    else:

        faltantes.append((esp, keywords))

        print(f"No se encontró match para: {esp} | probé: {keywords}")

print("\nResumen:")

print("Clases copiadas:", [c[1] for c in copiadas])

print("No encontradas:", faltantes)

print("Guardado en:", dataset_destino)

Ejemplo de clases en Training:
['Apple Red 1', 'Banana 1', 'Cantaloupe 2', 'Cocos 1', 'Granadilla 1', 'Kiwi 1', 'Maracuja 1', 'Peach 2', 'Pear Forelle 1', 'Pineapple Mini 1']
Copiada: 'apple red 1' -> 'Apple Red 1'
Copiada: 'banana' -> 'Banana 1'
Copiada: 'apple red 1' -> 'Apple Red 1'
Copiada: 'banana' -> 'Banana 1'
Copiada: 'melon 2' -> 'Cantaloupe 2'
Copiada: 'cocos' -> 'Cocos 1'
Copiada: 'melon 2' -> 'Cantaloupe 2'
Copiada: 'cocos' -> 'Cocos 1'
Copiada: 'granadilla' -> 'Granadilla 1'
Copiada: 'granadilla' -> 'Granadilla 1'
Copiada: 'kiwi' -> 'Kiwi 1'
Copiada: 'maracuja' -> 'Maracuja 1'
Copiada: 'kiwi' -> 'Kiwi 1'
Copiada: 'maracuja' -> 'Maracuja 1'
Copiada: 'peach' -> 'Peach 2'
Copiada: 'pear forelle' -> 'Pear Forelle 1'
Copiada: 'peach' -> 'Peach 2'
Copiada: 'pear forelle' -> 'Pear Forelle 1'
Copiada: 'pineapple' -> 'Pineapple Mini 1'

Resumen:
Clases copiadas: ['Apple Red 1', 'Banana 1', 'Cantaloupe 2', 'Cocos 1', 'Granadilla 1', 'Kiwi 1', 'Maracuja 1', 'Peach 2', 'Pear Forelle 1

In [20]:
import os

import shutil

import numpy as np

# === Ruta de tu dataset filtrado actual ===

dataset_filtrado = "./ProyectoFinal/Dataset_Filtrado"

# === Crear carpetas de destino ===

train_dir = os.path.join(dataset_filtrado, "train")

test_dir = os.path.join(dataset_filtrado, "test")

os.makedirs(train_dir, exist_ok=True)

os.makedirs(test_dir, exist_ok=True)

# === Proporción de división ===

train_split = 0.7  # 70% entrenamiento, 30% prueba

# === Recorrer cada clase ===

for clase in os.listdir(dataset_filtrado):

    clase_path = os.path.join(dataset_filtrado, clase)

    # Evitar las carpetas train y test ya creadas

    if os.path.isdir(clase_path) and clase not in ["train", "test"]:

        # Crear subcarpetas en train y test

        os.makedirs(os.path.join(train_dir, clase), exist_ok=True)

        os.makedirs(os.path.join(test_dir, clase), exist_ok=True)

        # Listar imágenes de la clase

        imagenes = [f for f in os.listdir(clase_path) if f.lower().endswith(('.jpg', '.png'))]

        np.random.shuffle(imagenes)

        # Dividir índices

        split_idx = int(len(imagenes) * train_split)

        train_imgs = imagenes[:split_idx]

        test_imgs = imagenes[split_idx:]

        # Copiar imágenes

        for img in train_imgs:

            shutil.copy(os.path.join(clase_path, img), os.path.join(train_dir, clase, img))

        for img in test_imgs:

            shutil.copy(os.path.join(clase_path, img), os.path.join(test_dir, clase, img))

        print(f"Clase {clase}: {len(train_imgs)} en train, {len(test_imgs)} en test")

print("\nDataset dividido en:")

print("Train ->", train_dir)

print("Test  ->", test_dir)

Clase Pear Forelle 1: 343 en train, 148 en test
Clase Kiwi 1: 228 en train, 98 en test
Clase Apple Red 1: 240 en train, 104 en test
Clase Cocos 1: 240 en train, 103 en test
Clase Cantaloupe 2: 240 en train, 104 en test
Clase Peach 2: 361 en train, 155 en test
Clase Granadilla 1: 240 en train, 103 en test
Clase Banana 1: 240 en train, 103 en test
Clase Maracuja 1: 240 en train, 103 en test
Clase Pineapple Mini 1: 241 en train, 104 en test

Dataset dividido en:
Train -> ./ProyectoFinal/Dataset_Filtrado/train
Test  -> ./ProyectoFinal/Dataset_Filtrado/test
Clase Peach 2: 361 en train, 155 en test
Clase Granadilla 1: 240 en train, 103 en test
Clase Banana 1: 240 en train, 103 en test
Clase Maracuja 1: 240 en train, 103 en test
Clase Pineapple Mini 1: 241 en train, 104 en test

Dataset dividido en:
Train -> ./ProyectoFinal/Dataset_Filtrado/train
Test  -> ./ProyectoFinal/Dataset_Filtrado/test


In [21]:
import os

import shutil

# Ruta a tu dataset filtrado

dataset_filtrado = "./ProyectoFinal/Dataset_Filtrado"

# Carpetas que quieres conservar

keep = {"train", "test"}

# Recorremos lo que hay en Dataset_Filtrado

for item in os.listdir(dataset_filtrado):

    item_path = os.path.join(dataset_filtrado, item)

    if os.path.isdir(item_path) and item not in keep:

        shutil.rmtree(item_path)  # elimina carpeta completa

        print(f"Eliminada carpeta: {item}")

print("\nLimpieza completa. Ahora solo tienes 'train/' y 'test/' dentro de Dataset_Filtrado.")

Eliminada carpeta: Pear Forelle 1
Eliminada carpeta: Kiwi 1
Eliminada carpeta: Apple Red 1
Eliminada carpeta: Cocos 1
Eliminada carpeta: Cantaloupe 2
Eliminada carpeta: Peach 2
Eliminada carpeta: Granadilla 1
Eliminada carpeta: Banana 1
Eliminada carpeta: Maracuja 1
Eliminada carpeta: Pineapple Mini 1

Limpieza completa. Ahora solo tienes 'train/' y 'test/' dentro de Dataset_Filtrado.
